In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium

%matplotlib inline
import matplotlib.pyplot as plt

from folium.plugins import FastMarkerCluster # python map data visualization tool

In [3]:
df = pd.read_csv("NBI_Colorado_2019.csv")
list(df.columns.values.tolist())

C:\Users\tobby\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (71,73,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['STATE_CODE_001',
 'STRUCTURE_NUMBER_008',
 'RECORD_TYPE_005A',
 'ROUTE_PREFIX_005B',
 'SERVICE_LEVEL_005C',
 'ROUTE_NUMBER_005D',
 'DIRECTION_005E',
 'HIGHWAY_DISTRICT_002',
 'COUNTY_CODE_003',
 'PLACE_CODE_004',
 'FEATURES_DESC_006A',
 'CRITICAL_FACILITY_006B',
 'FACILITY_CARRIED_007',
 'LOCATION_009',
 'MIN_VERT_CLR_010',
 'KILOPOINT_011',
 'BASE_HWY_NETWORK_012',
 'LRS_INV_ROUTE_013A',
 'SUBROUTE_NO_013B',
 'LAT_016',
 'LONG_017',
 'DETOUR_KILOS_019',
 'TOLL_020',
 'MAINTENANCE_021',
 'OWNER_022',
 'FUNCTIONAL_CLASS_026',
 'YEAR_BUILT_027',
 'TRAFFIC_LANES_ON_028A',
 'TRAFFIC_LANES_UND_028B',
 'ADT_029',
 'YEAR_ADT_030',
 'DESIGN_LOAD_031',
 'APPR_WIDTH_MT_032',
 'MEDIAN_CODE_033',
 'DEGREES_SKEW_034',
 'STRUCTURE_FLARED_035',
 'RAILINGS_036A',
 'TRANSITIONS_036B',
 'APPR_RAIL_036C',
 'APPR_RAIL_END_036D',
 'HISTORY_037',
 'NAVIGATION_038',
 'NAV_VERT_CLR_MT_039',
 'NAV_HORR_CLR_MT_040',
 'OPEN_CLOSED_POSTED_041',
 'SERVICE_ON_042A',
 'SERVICE_UND_042B',
 'STRUCTURE_KIND_043A',
 '

# Method to split DMS string into decimal, minutes, and seconds elements

In [12]:
# print(str(df["LONG_017"][0]))
# seconds = str(df["LONG_017"][0])[-4:]
# minutes = str(df["LONG_017"][0])[3:5]
# degrees = str(df["LONG_017"][0])[:3]
# print("degrees: " + degrees)
# print("minutes: " + minutes)
# print("seconds: " + seconds)

print(str(df["LAT_016"][0]))

original_dms = str(df["LAT_016"][0])
temp_dms_periods = ""
temp_dms = ""


minutes = str(df["LAT_016"][0])[-6:-4]
seconds = str(df["LAT_016"][0])[-4:]
seconds_periods = str(df["LAT_016"][0])[-4:-2] + "." + str(df["LAT_016"][0])[-2:]

temp_dms = minutes
temp_dms = temp_dms + seconds

if temp_dms in original_dms:
    degrees = original_dms.replace(temp_dms,'')

temp_dms = degrees
temp_dms = temp_dms + minutes
temp_dms = temp_dms + seconds


temp_dms_periods = degrees
temp_dms_periods = temp_dms_periods + "." + minutes
temp_dms_periods = temp_dms_periods + "." + seconds
    
# print("new_str: " + new_str)
print("temp_dms: " + temp_dms)
print("temp_dms_periods: " + temp_dms_periods)

print("degrees: " + degrees)
print("minutes: " + minutes)
print("seconds: " + seconds)

38263100
temp_dms: 38263100
temp_dms_periods: 38.26.3100
degrees: 38
minutes: 26
seconds: 3100


# Method to convert DMS to decimal coordinates, will need to do 360 - second coordinate to convert form West to East Longitude

In [25]:
def dms2dd(s):

    minutes = s[-6:-4]
    seconds = s[-4:]
    seconds_decimal = s[-4:-2] + "." +s[-2:]

    temp_dms = minutes
    temp_dms = temp_dms + seconds
    
    if temp_dms in s:
        degrees = s.replace(temp_dms,'')
    
    dd = float(degrees) + float(minutes)/60 + float(seconds_decimal)/(60*60);
    
    return dd
    
dms2dd(str(df["LONG_017"][0]))
dms2dd(str(df["LAT_016"][0]))

38.44194444444444

In [26]:
decimal_lats = []
decimal_longs = []

for index, row in df.iterrows():
    str_lat = str(row["LAT_016"])
    str_long = str(row["LONG_017"])
    
    if len(str_lat) < 8 or len(str_long) < 8:
        continue
    
    decimal_lat = dms2dd(str_lat)
    decimal_long = dms2dd(str_long)
    
    decimal_long = 360 - decimal_long
    
    decimal_lats.append(decimal_lat)
    decimal_longs.append(decimal_long)
    
decimal_lats
decimal_longs

df_new = pd.DataFrame()
df_new['decimal_lats'] = decimal_lats
df_new['decimal_longs'] = decimal_longs

df_new

,decimal_lats,decimal_longs
0,38.441944,252.444722
1,38.446944,252.444722
2,38.463889,252.450278
3,39.225556,252.189167
4,39.232139,252.186169
...,...,...
8757,39.760000,257.535917
8758,39.616842,257.599158
8759,39.626692,257.599183
8760,39.572217,257.639106


In [27]:
locations = df_new[['decimal_lats', 'decimal_longs']]
locationlist = locations.values.tolist()
locationlist[7]

[37.79356111111111, 254.1589111111111]

In [28]:
map = folium.Map(zoom_start=12)
for point in range(0, len(locationlist[:100])):
    folium.Marker(locationlist[point]).add_to(map)
map